# Import libs

In [1]:
from tokenizers import Tokenizer, pre_tokenizers, trainers, models
from datasets import load_dataset

ds = load_dataset("ncduy/mt-en-vi")

In [2]:
ds

DatasetDict({
    train: Dataset({
        features: ['en', 'vi', 'source'],
        num_rows: 2884451
    })
    validation: Dataset({
        features: ['en', 'vi', 'source'],
        num_rows: 11316
    })
    test: Dataset({
        features: ['en', 'vi', 'source'],
        num_rows: 11225
    })
})

In [3]:
# ds.remove_columns(["source"])

# Tokenize / Preprocessing

In [3]:
# word - based
# check if there is a tokenizer file
import os
if not os.path.exists("tokenizer_en.json") or not os.path.exists("tokenizer_vi.json"):
    tokenizer_en = Tokenizer(models.BPE(unk_token="<unk>"))
    tokenizer_vi = Tokenizer(models.BPE(unk_token="<unk>"))
    tokenizer_en.pre_tokenizer = pre_tokenizers.Whitespace()
    tokenizer_vi.pre_tokenizer = pre_tokenizers.Whitespace()
    trainer = trainers.BpeTrainer(
        vocab_size=100_000,
        min_frequency=2,
        special_tokens=["<pad>", "<unk>", "<bos>", "<eos>"],
    )
    # train tokenizer   
    tokenizer_en.train_from_iterator(ds["train"]["en"], trainer)
    tokenizer_vi.train_from_iterator(ds["train"]["vi"], trainer)
    # tokenizer
    tokenizer_en.save("tokenizer_en.json")
    tokenizer_vi.save("tokenizer_vi.json")

# Build vocabulary

In [3]:
from transformers import PreTrainedTokenizerFast

MAX_LEN = 50

# Load tokenizer
tokenizer_en = PreTrainedTokenizerFast(
    tokenizer_file="tokenizer_en.json",
    unk_token="<unk>",
    pad_token="<pad>",
    bos_token="<bos>",
    eos_token="<eos>",
)

tokenizer_vi = PreTrainedTokenizerFast(
    tokenizer_file="tokenizer_vi.json",
    unk_token="<unk>",
    pad_token="<pad>",
    bos_token="<bos>",
    eos_token="<eos>",
)

def preprocess_function(examples):
    src_texts = examples["en"]
    tgt_texts = ["<bos>" + sent + "<eos>" for sent in examples["vi"]]
    src_encodings = tokenizer_en(
        src_texts, padding="max_length", truncation=True, max_length=MAX_LEN
    )
    tgt_encodings = tokenizer_vi(
        tgt_texts, padding="max_length", truncation=True, max_length=MAX_LEN
    )

    return {
        "input_ids": src_encodings["input_ids"],
        "labels": tgt_encodings["input_ids"],
    }


preprocessed_ds = ds.map(preprocess_function, batched=True)

def is_valid_sample(sample):
    return any(token != 0 for token in sample["input_ids"])

preprocessed_ds = preprocessed_ds.filter(is_valid_sample)


c:\Users\Admin\miniconda3\envs\tf-gpu\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
preprocessed_ds['train'][20]

{'en': 'Smallpox also ravaged Mexico in the 1520s, killing 150,000 in Tenochtitlán alone, including the emperor, and Peru in the 1530s, aiding the European conquerors.',
 'vi': 'Bệnh đậu mùa cũng tàn phá México vào những năm 1520, chỉ riêng người Tenochtitlán đã có hơn 150.000 người chết, gồm cả quốc vương, và Peru vào những năm 1530, nhờ đó hỗ trợ cho những người châu Âu đi chinh phục.',
 'source': 'WikiMatrix v1',
 'input_ids': [85236,
  6881,
  35237,
  9659,
  6610,
  6613,
  7244,
  21390,
  15,
  10125,
  10993,
  15,
  7181,
  6610,
  56224,
  54428,
  8523,
  15,
  7452,
  6613,
  11694,
  15,
  6629,
  11891,
  6610,
  6613,
  7244,
  24843,
  15,
  33059,
  6613,
  8456,
  58201,
  17,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'labels': [2,
  9263,
  9329,
  7286,
  6600,
  8402,
  7268,
  10525,
  6521,
  6514,
  6495,
  28262,
  15,
  6617,
  7393,
  6485,
  27157,
  54477,
  6475,
  6463,
  6622,
  9556,
  17,
  7144,
  6485,
  6963,

# Modeling

In [4]:
import torch
import torch.nn as nn
from transformers import PreTrainedModel, PretrainedConfig

## GRU

In [8]:
class Seq2SeqRNNConfig(PretrainedConfig):
    def __init__(
        self,
        vocab_size_src=15000,
        vocab_size_tgt=15000,
        embedding_dim=256,
        hidden_size=256,
        drop_out=0.15,
    ):
        super().__init__()
        self.vocab_size_src = vocab_size_src
        self.vocab_size_tgt = vocab_size_tgt
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        self.drop_out = drop_out


class EncoderRNN(nn.Module):
    def __init__(self, input_size, embedding_dim, hidden_size, drop_out=0.15):
        super().__init__()
        self.embedding = nn.Embedding(
            input_size, embedding_dim
        )  # input_size = vn_vocab_size
        self.hidden_size = hidden_size
        self.gru = nn.GRU(embedding_dim, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(drop_out)

    def forward(self, x):
        embedded = self.embedding(x)
        embedded = self.dropout(embedded)
        output, hidden = self.gru(embedded)
        return output, hidden  # B x S x H, B x H


class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding_dim, output_size):
        super().__init__()
        self.embedding = nn.Embedding(
            output_size, embedding_dim
        )  # output_size = en_vocab_size
        self.gru = nn.GRU(embedding_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        embedded = self.embedding(x)
        output, hidden = self.gru(embedded, hidden)  # with hidden is h0
        output = self.fc(output)
        return output, hidden


class Seq2SeqRNNModel(PreTrainedModel):
    def __init__(self, config, tokenizer_en):
        super().__init__(config)
        self.encoder = EncoderRNN(
            config.vocab_size_src,
            config.embedding_dim,
            config.hidden_size,
            config.drop_out,
        )
        self.decoder = DecoderRNN(
            config.hidden_size, config.embedding_dim, config.vocab_size_tgt
        )
        self.BOS_IDX = tokenizer_en.bos_token_id
        self.loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer_en.pad_token_id)

    def forward(self, input_ids, labels):
        batch_size, seq_len = labels.shape  # get batch_size and seq_len
        decoder_input = torch.full((batch_size, 1), self.BOS_IDX, dtype=torch.long).to(input_ids.device)  # generate "<bos>" token for a batch
        encoder_output, decoder_hidden = self.encoder(input_ids) # _, h0
        decoder_outputs = []

        for i in range(seq_len - 1):
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)
            decoder_input = labels[:, i + 1].unsqueeze(1)  # shift left (teacher forcing)

        logits = torch.cat(decoder_outputs, dim=1)  # B x seq_len x Vocab
        loss = self.loss_fn(logits.permute(0, 2, 1), labels[:, 1:])
        return {"loss": loss, "logits": logits}


config = Seq2SeqRNNConfig(
    vocab_size_src=len(tokenizer_en), vocab_size_tgt=len(tokenizer_vi)
)
model = Seq2SeqRNNModel(config, tokenizer_en)

## Transformer

In [5]:
class Seq2SeqTransformerConfig(PretrainedConfig):
    model_type = "seq2seq_transformer"

    def __init__(
        self,
        vocab_size_src=100_000,
        vocab_size_tgt=100_000,
        d_model=512,
        num_heads=8,
        num_layers=12,
        max_seq_len=50,
        drop_out=0.1,
        **kwargs,  # This allows extra parameters for compatibility
    ): 
        super().__init__(**kwargs)  # Ensures Hugging Face can save/load config
        self.vocab_size_src = vocab_size_src
        self.vocab_size_tgt = vocab_size_tgt
        self.d_model = d_model
        self.num_heads = num_heads
        self.num_layers = num_layers
        self.max_seq_len = max_seq_len
        self.drop_out = drop_out


class Seq2SeqTransformerModel(PreTrainedModel):
    config_class = Seq2SeqTransformerConfig
    def __init__(self, config):
        super().__init__(config)
        self.embedding_src = nn.Embedding(config.vocab_size_src, config.d_model)
        self.embedding_tgt = nn.Embedding(config.vocab_size_tgt, config.d_model)

        self.position_embedding_src = nn.Embedding(config.max_seq_len, config.d_model)
        self.position_embedding_tgt = nn.Embedding(config.max_seq_len, config.d_model)

        self.transformer = nn.Transformer(
            d_model=config.d_model,
            nhead=config.num_heads,
            num_encoder_layers=config.num_layers,
            num_decoder_layers=config.num_layers,
            dropout=config.drop_out,
            batch_first=True,
        )
        self.generator = nn.Linear(config.d_model, config.vocab_size_tgt)
        self.loss_fn = nn.CrossEntropyLoss(ignore_index=0)

    def forward(self, input_ids, labels):
        tgt_input = labels[:, :-1]  # decoder input B x seq_len
        tgt_output = labels[:, 1:]  # decoder output B x seq_len

        batch_size, seq_len_src = input_ids.shape
        _, seg_len_tgt = tgt_input.shape

        # generate positional embedding
        src_positions = torch.arange(seq_len_src, device=input_ids.device).unsqueeze(
            0
        )  # 1 x seq
        tgt_positions = torch.arange(seg_len_tgt, device=labels.device).unsqueeze(
            0
        )  # 1 x seq
        
        # sum embedding
        # (B x seq) + (1 x seq) = embedded
        src_embedded = self.embedding_src(input_ids) + self.position_embedding_src(
            src_positions
        )  # B x seq
        tgt_embedded = self.embedding_tgt(tgt_input) + self.position_embedding_tgt(
            tgt_positions
        )  # B x seq

        # generate mask
        src_mask, tgt_mask, src_key_padding_mask, tgt_key_padding_mask = self.create_mask(
            input_ids, tgt_input
        )

        # output
        output = self.transformer(
            src=src_embedded,
            tgt=tgt_embedded,
            src_mask=src_mask,
            tgt_mask=tgt_mask,
            src_key_padding_mask=src_key_padding_mask,
            tgt_key_padding_mask=tgt_key_padding_mask,
        )  # shape (B, Seq, E_dim)
  
        logits = self.generator(output)  # (B, Seq, Vocab_size)
        loss = self.loss_fn(logits.permute(0, 2, 1), tgt_output)

        return {
            "loss": loss,
            "logits": logits,
        }

    # Define functions for inference phase
    def encode(self, src, src_mask, src_padding_mask):
        """
        Inference Encoder, this require a padding mask, if not, significant drop in performance.
        """
        _, src_len_src = src.shape
        src_positions = torch.arange(src_len_src, device=src.device).unsqueeze(0)
        src_embedded = self.embedding_src(src) + self.position_embedding_src(
            src_positions
        )
        return self.transformer.encoder(src_embedded, src_mask, src_padding_mask)

    def decode(self, tgt, encoder_output, causal_mask):
        """
        Inference Decoder, this require a causal mask for auto-regressive,\
        if not, significant drop in performance.\n
        Does not need a padding mask because the model want to predict an "eos" token
        """
        _, seq_len_tgt = tgt.shape
        tgt_positions = torch.arange(seq_len_tgt, device=tgt.device).unsqueeze(0)
        tgt_embedded = self.embedding_tgt(tgt) + self.position_embedding_tgt(
            tgt_positions
        )
        return self.transformer.decoder(tgt_embedded, encoder_output, causal_mask)

    def generate_square_subsequent_mask(self, sz, device):
        mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
        mask = (
            mask.float()
            .masked_fill(mask == 0, float("-inf")) # id nào bằng 0 thì chặn không cho tính attention
            .masked_fill(mask == 1, float(0.0))
        )
        return mask

    def create_mask(self, src, tgt):
        src_seq_len = src.shape[1]
        tgt_seq_len = tgt.shape[1]
        device = src.device
        tgt_mask = self.generate_square_subsequent_mask(tgt_seq_len, device).to(torch.bool)
        src_mask = torch.zeros((src_seq_len, src_seq_len), device=device).type(torch.bool)
        src_padding_mask = (src == 0) # id nào bằng 0 thì chặn không cho tính attention
        tgt_padding_mask = (tgt == 0)
        return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

config = Seq2SeqTransformerConfig(
    vocab_size_src=len(tokenizer_en),
    vocab_size_tgt=len(tokenizer_vi),
)
model_transformer = Seq2SeqTransformerModel(config)

# Testing

In [11]:
input_ids = torch.tensor([preprocessed_ds["train"][10]["input_ids"]])
labels = torch.tensor([preprocessed_ds["train"][10]["labels"]])
pred = model_transformer(input_ids=input_ids, labels=labels)


In [12]:
pred['logits'].shape

torch.Size([1, 49, 100000])

# Trainer

In [12]:
# Disable wandb
import os

os.environ["WANDB_DISABLED"] = "true"
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

# Training
training_args = Seq2SeqTrainingArguments(
    output_dir="./transformer-en-vi",
    logging_dir="logs",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=100,
    per_device_eval_batch_size=100,
    num_train_epochs=5,
    learning_rate=3e-05,
    save_total_limit=1,
    load_best_model_at_end=True,
    bf16=True,
    weight_decay=0.01,
    #report_to="wandb",
    gradient_accumulation_steps=4,
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [13]:
trainer = Seq2SeqTrainer(
    model=model_transformer,
    args=training_args,
    train_dataset=preprocessed_ds["train"],
    eval_dataset=preprocessed_ds["validation"],
)
trainer.train()

  0%|          | 0/36055 [00:00<?, ?it/s]

c:\Users\Admin\miniconda3\envs\tf-gpu\lib\site-packages\torch\nn\functional.py:5560: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


{'loss': 5.3254, 'grad_norm': 1.4536958932876587, 'learning_rate': 2.4e-05, 'epoch': 1.0}


  0%|          | 0/114 [00:00<?, ?it/s]

{'eval_loss': 4.299125671386719, 'eval_runtime': 8.0164, 'eval_samples_per_second': 1411.613, 'eval_steps_per_second': 14.221, 'epoch': 1.0}
{'loss': 4.1963, 'grad_norm': 1.3414915800094604, 'learning_rate': 1.8e-05, 'epoch': 2.0}


...

  0%|          | 0/114 [00:00<?, ?it/s]

{'eval_loss': 3.2314250469207764, 'eval_runtime': 7.7061, 'eval_samples_per_second': 1468.455, 'eval_steps_per_second': 14.794, 'epoch': 4.0}
{'loss': 3.4553, 'grad_norm': 1.2886652946472168, 'learning_rate': 0.0, 'epoch': 5.0}


  0%|          | 0/114 [00:00<?, ?it/s]

{'eval_loss': 3.1628687381744385, 'eval_runtime': 7.5341, 'eval_samples_per_second': 1501.969, 'eval_steps_per_second': 15.131, 'epoch': 5.0}
{'train_runtime': 27206.1952, 'train_samples_per_second': 530.109, 'train_steps_per_second': 1.325, 'train_loss': 4.07182592133546, 'epoch': 5.0}


TrainOutput(global_step=36055, training_loss=4.07182592133546, metrics={'train_runtime': 27206.1952, 'train_samples_per_second': 530.109, 'train_steps_per_second': 1.325, 'total_flos': 6.03894467216448e+17, 'train_loss': 4.07182592133546, 'epoch': 4.999826659733056})

In [16]:
torch.cuda.empty_cache()

In [16]:
trainer.evaluate(preprocessed_ds["test"])

  0%|          | 0/113 [00:00<?, ?it/s]

{'eval_loss': 3.1766247749328613,
 'eval_runtime': 9.5459,
 'eval_samples_per_second': 1175.904,
 'eval_steps_per_second': 11.838,
 'epoch': 4.999826659733056}

In [ ]:
torch.save(model_transformer.state_dict(), "./transformer-en-vi/model_weights.pth")

In [6]:
model_transformer.load_state_dict(torch.load("./transformer-en-vi/model_weights.pth", weights_only=True))

<All keys matched successfully>

In [24]:
model_transformer.save_pretrained("./transformer-en-vi/hub")
tokenizer_en.save_pretrained("./transformer-en-vi/hub/tokenizer_en")
tokenizer_vi.save_pretrained("./transformer-en-vi/hub/tokenizer_vi")

('./transformer-en-vi/hub/tokenizer_vi\\tokenizer_config.json',
 './transformer-en-vi/hub/tokenizer_vi\\special_tokens_map.json',
 './transformer-en-vi/hub/tokenizer_vi\\tokenizer.json')

In [ ]:
from huggingface_hub import HfApi

# Tạo repository trên Hugging Face
model_name = "binhphap5/en-vi-machine-translation"
api = HfApi()
api.create_repo(model_name, exist_ok=True) 

model_transformer.push_to_hub(model_name)
tokenizer_en.push_to_hub(model_name)  
tokenizer_vi.push_to_hub(model_name)  

# Inference

In [ ]:
import torch
import torch.nn.functional as F

def batch_beam_search_decode(model, src_sentences, tokenizer_en, tokenizer_vi, beam_width=5, max_len=50, temperature=1, device="cuda"):
    """
    Thực hiện beam search decode theo batch.
    """
    model.to(device)
    model.eval()
    bos_id = tokenizer_vi.bos_token_id
    eos_id = tokenizer_vi.eos_token_id

    # 1. Tokenize toàn bộ batch với padding cố định
    encoded = tokenizer_en.batch_encode_plus(
        src_sentences,
        add_special_tokens=True,
        max_length=max_len,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    src_tensor = encoded['input_ids'].to(device)  # shape: (B, max_len)
    
    # Tạo src_mask và src_padding_mask 
    B, src_seq_len = src_tensor.shape
    src_mask = torch.zeros((src_seq_len, src_seq_len), dtype=torch.bool, device=device)
    src_padding_mask = (src_tensor == 0)  # True cho các vị trí padding

    # 2. Tính encoder output cho toàn batch
    with torch.no_grad():
        encoder_output = model.encode(src_tensor, src_mask, src_padding_mask)  
        # encoder_output shape: (B, src_seq_len, hidden_dim)

    # 3. Khởi tạo beams cho từng câu
    # beams: (B, beam_width, current_seq_len), ban đầu current_seq_len=1, chỉ chứa <bos>
    beams = torch.full((B, 1, 1), bos_id, dtype=torch.long, device=device)
    # scores: (B, beam_width), ban đầu là 0
    beam_scores = torch.zeros(B, 1, device=device)
    
    # Tạo danh sách đánh dấu beam nào đã kết thúc (mỗi câu)
    complete_beams = [ [] for _ in range(B) ]
    
    # Duyệt từng bước sinh token cho đến max_len
    for step in range(max_len - 1):  # đã có 1 token ban đầu
        B_current, beam_num, seq_len = beams.shape  # B_current == B
        # Flatten beams: (B * beam_width, seq_len)
        flat_beams = beams.view(B * beam_num, seq_len)
        
        # Tạo causal mask cho decoder: (seq_len, seq_len)
        # Giả sử mô hình có hàm generate_square_subsequent_mask
        causal_mask = model.generate_square_subsequent_mask(seq_len, device)  # shape: (seq_len, seq_len)
        
        # Lặp lại encoder output cho mỗi beam:
        # encoder_output: (B, src_seq_len, hidden_dim) -> (B, 1, src_seq_len, hidden_dim) -> (B, beam_width, src_seq_len, hidden_dim)
        # Sau đó flatten thành (B * beam_width, src_seq_len, hidden_dim)
        repeated_encoder_output = encoder_output.unsqueeze(1).repeat(1, beam_num, 1, 1).view(B * beam_num, src_seq_len, -1)
        
        #  Tính decoder output cho từng beam (batch mode)
        with torch.no_grad():
            decoder_output = model.decode(flat_beams, repeated_encoder_output, causal_mask)
            # decoder_output shape: (B * beam_width, seq_len, hidden_dim)
            logits = model.generator(decoder_output[:, -1, :])  # lấy logit của token mới nhất: (B * beam_width, vocab_size)
        
        # Áp dụng temperature và tính log softmax
        logits = logits / temperature
        log_probs = F.log_softmax(logits, dim=-1)  # (B * beam_width, vocab_size)
        
        # Reshape để tính điểm cho từng beam riêng biệt
        log_probs = log_probs.view(B, beam_num, -1)  # (B, beam_width, vocab_size)
        
        # Cộng dồn điểm của các beam hiện tại với log_probs của token tiếp theo
        # beam_scores: (B, beam_width) -> unsqueeze(-1): (B, beam_width, 1)
        total_scores = beam_scores.unsqueeze(-1) + log_probs  # (B, beam_width, vocab_size)
        
        # Reshape lại để chọn top beam_width cho mỗi câu: (B, beam_width * vocab_size)
        total_scores = total_scores.view(B, -1)
        # Lấy top beam_width chỉ số và điểm
        topk_scores, topk_indices = total_scores.topk(beam_width, dim=-1)  # mỗi câu: (beam_width,)
        
        # Xác định beam index cũ và token mới được chọn
        beam_indices = topk_indices // log_probs.size(-1)  # (B, beam_width)
        token_indices = topk_indices % log_probs.size(-1)    # (B, beam_width)
        
        # Cập nhật các beams mới theo beam_indices và token_indices
        new_beams = []
        new_beam_scores = []
        for i in range(B):
            beams_i = beams[i]  # (beam_num, seq_len)
            new_beams_i = []
            new_scores_i = []
            for j in range(beam_width):
                prev_beam = beams_i[beam_indices[i, j]]
                new_token = token_indices[i, j].unsqueeze(0)
                new_seq = torch.cat([prev_beam, new_token])
                new_beams_i.append(new_seq.unsqueeze(0))
                new_scores_i.append(topk_scores[i, j].unsqueeze(0))
            # Coi new_beams_i là tensor (beam_width, seq_len_new)
            new_beams.append(torch.cat(new_beams_i, dim=0).unsqueeze(0))
            new_beam_scores.append(torch.cat(new_scores_i, dim=0).unsqueeze(0))
        # Ghép lại theo batch
        beams = torch.cat(new_beams, dim=0)  # (B, beam_width, seq_len_new)
        beam_scores = torch.cat(new_beam_scores, dim=0)  # (B, beam_width)
        
        # Kiểm tra các beam đã kết thúc (nếu token cuối là eos)
        # Lưu lại các beam đã hoàn thành và đặt điểm của chúng thành -inf để không chọn tiếp
        beams_list = []
        scores_list = []
        for i in range(B):
            beams_i = beams[i]
            scores_i = beam_scores[i]
            ongoing_beams = []
            ongoing_scores = []
            for j in range(beam_width):
                if beams_i[j, -1].item() == eos_id:
                    complete_beams[i].append((beams_i[j], scores_i[j]))
                else:
                    ongoing_beams.append(beams_i[j].unsqueeze(0))
                    ongoing_scores.append(scores_i[j].unsqueeze(0))
            # Nếu không còn beam nào chưa kết thúc, giữ lại beam tốt nhất (để vòng lặp không dừng)
            if len(ongoing_beams) == 0:
                ongoing_beams = [beams_i[0].unsqueeze(0)]
                ongoing_scores = [scores_i[0].unsqueeze(0)]
            beams_list.append(torch.cat(ongoing_beams, dim=0))
            scores_list.append(torch.cat(ongoing_scores, dim=0))
        # Cập nhật beams và beam_scores sau khi lọc theo từng câu
        # Lưu ý: số beam có thể khác nhau giữa các câu, ta cần pad lại về beam_width
        new_beams = []
        new_scores = []
        for i in range(B):
            cur_beams = beams_list[i]
            cur_scores = scores_list[i]
            cur_beam_num = cur_beams.shape[0]
            if cur_beam_num < beam_width:
                # Nếu số beam ít hơn beam_width, ta có thể pad thêm các giá trị rất thấp để tránh bị chọn
                pad_num = beam_width - cur_beam_num
                pad_seq = cur_beams[0].unsqueeze(0).repeat(pad_num, 1)
                pad_scores = torch.full((pad_num,), -1e9, device=device)
                cur_beams = torch.cat([cur_beams, pad_seq], dim=0)
                cur_scores = torch.cat([cur_scores, pad_scores], dim=0)
            new_beams.append(cur_beams.unsqueeze(0))
            new_scores.append(cur_scores.unsqueeze(0))
        beams = torch.cat(new_beams, dim=0)  # (B, beam_width, seq_len_new)
        beam_scores = torch.cat(new_scores, dim=0)  # (B, beam_width)
        
        # Nếu với mỗi câu, tất cả các beam đều đã kết thúc, ta có thể dừng sớm
        if all(len(complete_beams[i]) >= beam_width for i in range(B)):
            break

    # Chọn kết quả tốt nhất cho mỗi câu
    final_translations = []
    for i in range(B):
        if complete_beams[i]:
            # Lấy beam có điểm cao nhất
            best_beam = max(complete_beams[i], key=lambda x: x[1])[0]
        else:
            best_beam = beams[i][0]
        # Giải mã token sang câu
        translation = tokenizer_vi.decode(best_beam.tolist(), skip_special_tokens=True)
        final_translations.append(translation)
        
    return final_translations

# Example usage:
src_sentences = ["A cat is going to the moon with its ship.",
                "Everyone is happy because the sun is shining.",
                "I am no longer a child, but I still love to play with toys, strange right?",]
translations = batch_beam_search_decode(model_transformer, src_sentences, tokenizer_en, tokenizer_vi, beam_width=5, max_len=50, temperature=1, device="cuda")
# free up VRAM
torch.cuda.empty_cache()
print("batch translation:")
for idx, trans in enumerate(translations):
    print(f"{idx+1}: {trans}")

batch translation:
1: Con mèo sẽ đến mặt trăng với con tàu.
2: Mọi người đều hạnh phúc vì mặt trời mọc.
3: Tôi không còn là một đứa trẻ, nhưng tôi vẫn yêu chơi với đồ chơi, lạ phải không?


# BLEU Score

In [17]:
import torch
import torch.nn.functional as F
from nltk.translate.bleu_score import corpus_bleu

def compute_bleu_score(model, test_dataset, tokenizer_en, tokenizer_vi, beam_width=5, max_len=50, temperature=1, device="cuda", batch_size=128):
    """
    Tính BLEU score cho tập test bằng cách chia thành các batch nhỏ.
    
    test_dataset: dict có ít nhất 2 trường 'en' và 'vi'
    """
    src_sentences = test_dataset['en']  # danh sách câu tiếng Anh
    target_sentences = test_dataset['vi']  # danh sách câu tiếng Việt tham chiếu

    all_predictions = []
    n_samples = len(src_sentences)
    # Chia tập test thành các batch nhỏ theo batch_size
    for i in range(0, n_samples, batch_size):
        batch_src = src_sentences[i:i+batch_size]
        # Gọi hàm dịch theo batch đã định nghĩa
        batch_predictions = batch_beam_search_decode(model, batch_src, tokenizer_en, tokenizer_vi,
                                                       beam_width=beam_width, max_len=max_len,
                                                       temperature=temperature, device=device)
        all_predictions.extend(batch_predictions)
        print(f"Processed {min(i+batch_size, n_samples)} / {n_samples}")

    # Chuẩn hóa câu: tách token theo khoảng trắng
    # Yêu cầu của corpus_bleu: danh sách tham chiếu cho mỗi câu dưới dạng list[list[str]]
    references = [[ref.split()] for ref in target_sentences]
    hypotheses = [pred.split() for pred in all_predictions]

    bleu_score = corpus_bleu(references, hypotheses)
    return bleu_score

# Giả sử preprocessed_ds['test'] chứa các trường 'en', 'vi'
bleu = compute_bleu_score(model_transformer, preprocessed_ds['test'], tokenizer_en, tokenizer_vi,
                          beam_width=5, max_len=50, temperature=1, device="cuda", batch_size=128)
# free up VRAM
torch.cuda.empty_cache()
print(f"\nBLEU score trên tập test: {bleu:.4f}")

Processed 128 / 11225
Processed 256 / 11225
Processed 384 / 11225
Processed 512 / 11225
Processed 640 / 11225
Processed 768 / 11225
Processed 896 / 11225
Processed 1024 / 11225
Processed 1152 / 11225
Processed 1280 / 11225
Processed 1408 / 11225
Processed 1536 / 11225
Processed 1664 / 11225
Processed 1792 / 11225
Processed 1920 / 11225
Processed 2048 / 11225
Processed 2176 / 11225
Processed 2304 / 11225
Processed 2432 / 11225
Processed 2560 / 11225
Processed 2688 / 11225
Processed 2816 / 11225
Processed 2944 / 11225
Processed 3072 / 11225
Processed 3200 / 11225
Processed 3328 / 11225
Processed 3456 / 11225
Processed 3584 / 11225
Processed 3712 / 11225
Processed 3840 / 11225
Processed 3968 / 11225
Processed 4096 / 11225
Processed 4224 / 11225
Processed 4352 / 11225
Processed 4480 / 11225
Processed 4608 / 11225
Processed 4736 / 11225
Processed 4864 / 11225
Processed 4992 / 11225
Processed 5120 / 11225
Processed 5248 / 11225
Processed 5376 / 11225
Processed 5504 / 11225
Processed 5632 / 1